In [110]:
# Load packages
import numpy as np
import pandas as p
from datetime import datetime

# 1. Data Preprocessing

## 1.1 Consolidate Dataset

In [111]:
# Load previously saved CIBC tweets and app reviews
df_tweets = pd.read_csv(r'cibc_tweets.csv',encoding='iso-8859-1')
df_app = pd.read_excel('CIBC_App_Reviews.xlsx')
# Check the top records
print(df_tweets.head())
print(df_app.head())

             user                   fullname                   id  \
0   Play2LearnEdu  Play2Learn Education®²?¹?  1096558968581742592   
1   Play2LearnEdu  Play2Learn Education®²?¹?  1096557715768279045   
2  FiebigerWriter            Andrew Fiebiger  1096555080285732864   
3     langaraCCDC       Langara Co-op/Career  1096551195307429890   
4  montrealinform          Montreal Informer  1096546075123568641   

                                          url            timestamp  \
0   /Play2LearnEdu/status/1096558968581742592  2019-02-15 23:56:45   
1   /Play2LearnEdu/status/1096557715768279045  2019-02-15 23:51:47   
2  /FiebigerWriter/status/1096555080285732864  2019-02-15 23:41:18   
3     /langaraCCDC/status/1096551195307429890  2019-02-15 23:25:52   
4  /montrealinform/status/1096546075123568641  2019-02-15 23:05:31   

                                                text  replies  retweets  \
0  FLL #team42177 #supernova team member Davis pr...        0         0   
1  FLL #team42

Now we are going to extract the columns we need for the sentiment analysis.


In [112]:
# Check the columns
print('-- Columns for df_tweets--\n')
print(df_tweets.dtypes)
print('\n-- Columns for df_app--\n')
print(df_app.dtypes)


-- Columns for df_tweets--

user         object
fullname     object
id            int64
url          object
timestamp    object
text         object
replies       int64
retweets      int64
likes         int64
html         object
dtype: object

-- Columns for df_app--

App Name                      object
App Store                     object
App                           object
Store                         object
App ID                        object
Review ID                     object
Country                       object
Version                       object
Rating                         int64
Date                  datetime64[ns]
Author                        object
Subject                       object
Body                          object
Translated Subject            object
Translated Body               object
Sentiment                     object
Device                       float64
Language                      object
OS Version                   float64
Reply URL                    

In the df_tweets, we are going to keep timestamp and text. Then, we will remove duplicates.

In [113]:
# Keep the timestamp and text columns
df_tweets_cleaned = df_tweets[['timestamp','text']]
print(df_tweets_cleaned.head(10))
print('\nTotal Number of Original Tweets: {}\n'.format(len(df_tweets_cleaned)))

# Convert them to lowercase
df_tweets_cleaned['text'] = df_tweets_cleaned['text'].str.lower()
print(df_tweets_cleaned.head(10))

# Remove duplicates
df_tweets_cleaned = df_tweets_cleaned.drop_duplicates(subset=['text'])
print('\nTotal Number of De-duplicated Tweets: {}\n'.format(len(df_tweets_cleaned)))
print(df_tweets_cleaned.head(20))

# Keep tweets that contain 'cibc'
df_tweets_cleaned = df_tweets_cleaned[df_tweets_cleaned['text'].str.contains('cibc')]
print('\nTotal Number of De-duplicated Tweets that contain \'cibc\': {}\n'.format(len(df_tweets_cleaned)))

# Format the timestamp to date
df_tweets_cleaned['date'] = [datetime.strptime(d[:10], '%Y-%m-%d').date() for d in df_tweets_cleaned['timestamp']]

# Keep only date and text columns and rename them
df_tweets_cleaned = df_tweets_cleaned[['date','text']]
df_tweets_cleaned.columns = ['Date','Comments']


df_tweets_cleaned.head(n=10)

             timestamp                                               text
0  2019-02-15 23:56:45  FLL #team42177 #supernova team member Davis pr...
1  2019-02-15 23:51:47  FLL #team42177 #supernova team member Davis pr...
2  2019-02-15 23:41:18  Hey @cibc\nI would like to recommend the idea ...
3  2019-02-15 23:25:52  We hosted the 9th PDD Morning Mingle where Pos...
4  2019-02-15 23:05:31  CIBC Innovation Banking hires for Montréal, Me...
5  2019-02-15 23:00:51  Thank you so much to Virden CIBC for donating ...
6  2019-02-15 22:51:27  Thank you so much to Virden CIBC for donating ...
7  2019-02-15 22:39:55  It was a pleasant surprise to know that #cibc ...
8  2019-02-15 22:29:18  CIBC Innovation Banking hires for Montréal, Me...
9  2019-02-15 22:21:14  CIBC has been shit with my RRSP , moved them t...

Total Number of Original Tweets: 24081

             timestamp                                               text
0  2019-02-15 23:56:45  fll #team42177 #supernova team member davis pr.

C:\Users\jwchen69\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,Date,Comments
0,2019-02-15,fll #team42177 #supernova team member davis pr...
1,2019-02-15,fll #team42177 #supernova team member davis pr...
2,2019-02-15,hey @cibc\ni would like to recommend the idea ...
3,2019-02-15,we hosted the 9th pdd morning mingle where pos...
4,2019-02-15,"cibc innovation banking hires for montréal, me..."
5,2019-02-15,thank you so much to virden cibc for donating ...
6,2019-02-15,thank you so much to virden cibc for donating ...
7,2019-02-15,it was a pleasant surprise to know that #cibc ...
8,2019-02-15,"cibc innovation banking hires for montréal, me..."
9,2019-02-15,"cibc has been shit with my rrsp , moved them t..."


Now we will be cleaning the app reviews data and then merge the two datasets.

In [114]:
# Now select only English reviews.
print('--Total Number of App Reivews: {} --\n'.format(len(df_app)))
df_app_cleaned = df_app[df_app['Language'] == 'English']
print('--Total Number of English App Reivews: {} --\n'.format(len(df_app_cleaned)))

# Then select only Date and Body column
df_app_cleaned = df_app_cleaned[['Date','Body']]
df_app_cleaned.columns = ['Date','Comments']
print(df_app_cleaned.head())

df_app_cleaned.dtypes

--Total Number of App Reivews: 7738 --

--Total Number of English App Reivews: 7400 --

        Date                                           Comments
0 2019-03-20        This app is as crappy as the bank that made
1 2019-03-19  The credit score calculator has been down for ...
2 2019-03-17  I stopped banking with CIBC due to another ban...
4 2019-03-15  I have to change my password every time I try ...
5 2019-03-14  Can’t log out. Or is very cumbersome to log ou...


Date        datetime64[ns]
Comments            object
dtype: object

In [115]:
# Appending two dataframes
df_combined = pd.concat([df_tweets_cleaned, df_app_cleaned], ignore_index=True)
print(df_combined.head())
print('\n--Total Number of De-duplicated Tweets that contain \'cibc\': {} --\n'.format(len(df_tweets_cleaned)))
print('\n--Total Number of English App Reivews: {} --\n'.format(len(df_app_cleaned)))
print('\n--Total Number of Comments: {} --'.format(len(df_combined)))

         Date                                           Comments
0  2019-02-15  fll #team42177 #supernova team member davis pr...
1  2019-02-15  fll #team42177 #supernova team member davis pr...
2  2019-02-15  hey @cibc\ni would like to recommend the idea ...
3  2019-02-15  we hosted the 9th pdd morning mingle where pos...
4  2019-02-15  cibc innovation banking hires for montréal, me...

--Total Number of De-duplicated Tweets that contain 'cibc': 15630 --


--Total Number of English App Reivews: 7400 --


--Total Number of Comments: 23030 --


Afert consolidating the dataset we end up with **23,030** reviews to work with for CIBC.